In [4]:
!pip install requests pandas sqlalchemy
!pip install --user psycopg2-binary
!pip install pymysql


In [5]:
import os
import requests
import pandas as pd
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
from datetime import date

In [31]:
API_KEY = "API_KEY" 
CITY = "Pune"

In [32]:
USER = "root"
PASSWORD = "PASSWORD" 
HOST = "127.0.0.1" 
PORT = 3306
DB = "dma"

In [33]:
from datetime import date
from urllib.parse import quote_plus
import requests

MYSQL_CONN_STRING = f"mysql+pymysql://{USER}:{quote_plus(PASSWORD)}@{HOST}:{PORT}/{DB}"

def fetch_weather(api_key: str, city: str) -> dict:
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"q": city, "appid": api_key.strip(), "units": "metric"}

    r = requests.get(url, params=params, timeout=20)
    if r.status_code != 200:
        try:
            print("OpenWeather error payload:", r.json())
        except Exception:
            print("OpenWeather non-JSON response:", r.text)
        r.raise_for_status()

    data = r.json()
    return {
        "weather_date": date.today(),
        "city": city,
        "temp_c": data["main"]["temp"],
        "humidity": data["main"]["humidity"],
        "description": data["weather"][0]["description"],
    }


In [36]:
from sqlalchemy import create_engine, text
import pandas as pd

def ensure_weather_table(engine) -> None:
    ddl = """
    CREATE TABLE IF NOT EXISTS weather (
        weather_date DATE NOT NULL,
        city VARCHAR(100) NOT NULL,
        temp_c DECIMAL(5,2),
        humidity INT,
        description VARCHAR(255)
    )
    """
    with engine.begin() as conn:
        conn.execute(text(ddl))


def store_weather_to_db(weather_data: dict, conn_string: str) -> None:

    engine = create_engine(conn_string, pool_pre_ping=True)
    ensure_weather_table(engine)
    try:
        df = pd.DataFrame([weather_data])
        df.to_sql("weather", engine, if_exists="append", index=False)
        print(f"Weather data for {weather_data['city']} stored successfully in MySQL.")
    finally:
        engine.dispose()


def main():
    weather = fetch_weather(API_KEY, CITY)
    store_weather_to_db(weather, MYSQL_CONN_STRING)


if __name__ == "__main__":
    main()


Weather data for Pune stored successfully in MySQL.


In [37]:
import pandas as pd
from sqlalchemy import create_engine

# Connect to MySQL
engine = create_engine(MYSQL_CONN_STRING)

# Read the weather table into a DataFrame
df = pd.read_sql("SELECT * FROM weather", engine)

# Display the table
df


,weather_date,city,temp_c,humidity,description
0,2025-09-23,Pune,23.0,89,broken clouds
1,2025-09-23,Pune,23.0,89,broken clouds
